In [5]:
#r "nuget:Apache.Arrow"

Installed Packages Apache.Arrow, 11.0.0

In [6]:
using System;
using System.IO;
using System.Threading.Tasks;
using Apache.Arrow;
using Apache.Arrow.Ipc;
using Apache.Arrow.Types;

In [7]:
void CreateRandomFile(string path)
{
            // Generate random data
        var random = new Random();
        int numRows = 10;

        var names = new string[] { "Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Hank", "Iris", "John" };
        var nameBuilder = new StringArray.Builder();
        var valueBuilder = new FloatArray.Builder();

        for (int i = 0; i < numRows; i++)
        {
            nameBuilder.Append(names[random.Next(names.Length)]);
            valueBuilder.Append((float)random.NextDouble() * 100);
        }

        // Create a RecordBatch with the generated data
        var schema = new Schema.Builder()
            .Field(f => f.Name("name").DataType(StringType.Default))
            .Field(f => f.Name("value").DataType(FloatType.Default))
            .Build();

        var recordBatch = new RecordBatch(
            schema,
            new IArrowArray[] { nameBuilder.Build(), valueBuilder.Build() },
            numRows
        );

        // Write the RecordBatch to an Arrow IPC file
        using var stream = File.OpenWrite(path);
        using var writer = new ArrowStreamWriter(stream, schema);
        
        writer.WriteRecordBatch(recordBatch);

        Console.WriteLine($"Random Arrow IPC file created at: {path}");
}

In [8]:
public IEnumerable<object> ReadFile(string path)
{
        // Read the Arrow IPC file into a RecordBatch
        RecordBatch recordBatch;
        using (var stream = File.OpenRead(path))
        {
            var reader = new ArrowStreamReader(stream);
            recordBatch = reader.ReadNextRecordBatch();
        }

        // Access columns by name
        StringArray nameColumn = recordBatch.Column(recordBatch.Schema.GetFieldIndex("name")) as StringArray;
        FloatArray valueColumn = recordBatch.Column(recordBatch.Schema.GetFieldIndex("value")) as FloatArray;

        // Print the contents of the columns
        var ret = new List<object>();
        for (int i = 0; i < recordBatch.Length; i++)
        {
            ret.Add((Name: nameColumn.GetString(i), Value: valueColumn.GetValue(i)));
        }
        return ret;
}

In [17]:
var path = "/home/aron/ArrowPlayground/random_arrow_ipc_file.arrow";
CreateRandomFile(path);
ReadFile(path)

Random Arrow IPC file created at: /home/aron/ArrowPlayground/random_arrow_ipc_file.arrow


index value 0 (Grace, 1.7448843) Item1 Grace Item2 1.7448843 1 (Grace, 90.35024) Item1 Grace Item2 90.35024 2 (Iris, 8.6012125) Item1 Iris Item2 8.6012125 3 (Eve, 78.174675) Item1 Eve Item2 78.174675 4 (David, 50.894314) Item1 David Item2 50.894314 5 (Eve, 62.78351) Item1 Eve Item2 62.78351 6 (John, 13.651877) Item1 John Item2 13.651877 7 (Hank, 6.3881106) Item1 Hank Item2 6.3881106 8 (Frank, 25.696833) Item1 Frank Item2 25.696833 9 (David, 34.122314) Item1 David Item2 34.122314